### Проект по SQL

<b>Описание:</b>
#### Компания решила быть на волне и купила крупный сервис для чтения книг по подписке.  Первая задача — проанализировать базу данных.</br>
<br><b>Цель:</b></br>
<br>Сформулировать ценностное предложение для нового продукта.</br>
<br></br>

#### Описание данных
##### Таблица books
###### Содержит данные о книгах:
- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.
##### Таблица authors
###### Содержит данные об авторах:
- `author_id` — идентификатор автора;
- `author` — имя автора.
##### Таблица publishers
###### Содержит данные об издательствах:
- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;
##### Таблица ratings
###### Содержит данные о пользовательских оценках книг:
- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.
##### Таблица reviews
###### Содержит данные о пользовательских обзорах:
- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя автора обзора;
- `text` — текст обзора.


#### Схема данных

![picture_of_bird](https://pictures.s3.yandex.net/resources/scheme_1589269096.png)

### Загрузка данных и изучение общей информации

#### Импортируем необходимые библиотеки для работы

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [2]:
#Напишем функцию выводящую результат SQL-запроса:
def sql_result(query):
    return pd.io.sql.read_sql(query, con = engine)

#### Смотрим на содержание таблиц

In [3]:
query  = '''
        SELECT * 
        FROM books
        LIMIT 5
        '''
sql_result(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
query ='''
        SELECT * 
        FROM authors
        LIMIT 5
        '''
sql_result(query)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
query = '''
        SELECT * 
        FROM publishers
        LIMIT 5
        '''
sql_result(query)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [6]:
query = '''
        SELECT * 
        FROM ratings
        LIMIT 5
        '''
sql_result(query)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [7]:
query = '''
        SELECT * 
        FROM reviews
        LIMIT 5
        '''
sql_result(query)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


### Исследовательский анализ

#### Посчитайте, сколько книг вышло после 1 января 2000 года;

In [8]:
query = '''
        SELECT COUNT(DISTINCT book_id) 
        FROM books 
        WHERE publication_date > '2000-01-01' 
        '''
sql_result(query)

,count
0,819


<div class="alert alert-info"> Всего вышло 786 книг после 1 января 2000 года.</div>

#### Для каждой книги посчитайте количество обзоров и среднюю оценку;

In [9]:
query = '''
        SELECT books.title,
        COUNT(distinct review_id) as review_cnt,
        AVG(rating) AS avg_rating
        
        FROM books
        LEFT JOIN reviews ON reviews.book_id = books.book_id
        LEFT JOIN ratings ON ratings.book_id = books.book_id
        GROUP BY books.title
        ORDER BY 
        COUNT(reviews.review_id) DESC
        '''
sql_result(query)

,title,review_cnt,avg_rating
0,Twilight (Twilight #1),7,3.662500
1,The Hobbit or There and Back Again,6,4.125000
2,The Catcher in the Rye,6,3.825581
3,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
...,...,...,...
994,Disney's Beauty and the Beast (A Little Golden...,0,4.000000
995,Essential Tales and Poems,0,4.000000
996,Leonardo's Notebooks,0,4.000000
997,The Natural Way to Draw,0,3.000000


<div class="alert alert-info">Вывел топ 10 книг и отсортировал по количеству обзоров. Безоговорочным лидером является книга Twilight (Twilight #1) с 7 просмотрами.</div>

#### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

In [10]:
query = '''
                   SELECT p.publisher,
                   COUNT(b.book_id) as count_books
                   FROM books b
                   LEFT JOIN publishers p ON b.publisher_id = p.publisher_id 
                   WHERE num_pages > 50
                   GROUP BY 1
                   ORDER BY count_books DESC 
                   LIMIT 5
                   '''
sql_result(query)

,publisher,count_books
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Bantam,19


<div class="alert alert-info">Наибольшее число книг толще 50 страниц выпустило издательство Penguin Books - 42 книги.</div>

#### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [11]:
query = ''' WITH author as(
                   SELECT a.author_id ,
                   a.author as имя_автора
                   
                   FROM authors a
                   LEFT JOIN books b ON a.author_id = b.author_id
                   GROUP BY 1,2),
                   
                   rating as(SELECT b.author_id, 
                   COUNT(r.rating_id) as количество_оценок,
                   ROUND(avg(r.rating),2) as средний_рейтинг
                   
                   FROM books b
                   LEFT JOIN ratings r on b.book_id = r.book_id
                   GROUP BY 1
                   HAVING count(rating_id) >= 50
                   ORDER BY количество_оценок desc
                   )
                   
                   SELECT author.имя_автора,
                   rating.количество_оценок,
                   rating.средний_рейтинг
                   FROM author 
                   JOIN rating on author.author_id = rating.author_id
                   GROUP BY 1,2,3
                   ORDER BY 2 DESC
                   LIMIT 5
                   '''

sql_result(query)

,имя_автора,количество_оценок,средний_рейтинг
0,J.K. Rowling/Mary GrandPré,312,4.29
1,J.R.R. Tolkien,166,4.24
2,Stephenie Meyer,160,3.66
3,Dan Brown,143,3.74
4,Nicholas Sparks,111,3.88


<div class="alert alert-info">Автором с самой высокой средней оценкой книг — с учитом книги только с 50-ю и более оценками, стала J.K. Rowling/Mary GrandPré	, её рейтинг равен 4,29</div>

#### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [12]:
query = ''' SELECT
                                     ROUND(AVG(review_cnt),2) as среднее_количество_обзоров
            
                                     FROM (SELECT
                                     COUNT(review_id) as review_cnt
                  
                                     FROM reviews
                                     WHERE username IN(SELECT username
                  
                                                       FROM ratings
                                                       GROUP BY username
                                                       HAVING COUNT(ratings) > 50)
                                     GROUP BY username) as sub_review_cnt
                                 '''
sql_result(query)

,среднее_количество_обзоров
0,24.33


<div class="alert alert-info">Среднее количество обзоров от пользователей, которые поставили больше 50 оценок, равно 24.33</div>

Благодарю за совместную работу и хочу поздравить, ты большой молодец, что прошёл такой путь и освоил программу, надеюсь у тебя сбудутся все намеченные планы.

У Практикума развивается сообщество выпускников, так что рекомендую про него не забывать, в случае необходимости.


Понравилось, что ты переписал блок для получения первичного анализа по датафреймам, мне больше нравится функции, чтобы получался не такой большой блок вывода по кода, а потом как раз можно вставить краткий вывод в `markdown` ячейке. Но вообще автоматизировать повторяющиеся задачи - полезное стремление аналитика и я хочу порекламировать библиотеку `pandas profiling`, она как раз для первичного анализа). По визуализациям ниже предлагаю набор различных способов сделать их информативнее, там и возможность подписывать столбцы на столбчатых диаграммах и ещё всякое. Желаю удачи!)

Оформление презентаций дело субъективное и зачастую в разных компаниях имеются свои требования, но вот видео от Яндекса по созданию презентаций https://www.youtube.com/watch?v=S0r0fMJa9eA, а ещё на платформе практикума открылся бесплатный курс по созданию презентаций, рекомендую.

Для более глубокого изучения tableau, в случае необходимости рекомендую материал https://tableau.pro/m01, этот курс делает классная команда, у них бывают и другие интересные мини курсы. Но в последнее время российские компании уходят от tableau или уже ушли, поэтому возможно придётся познакомиться с другими инструментами.

Задачи на статистику или вокруг неё часто возникают на собеседовании, для укрепления этого фундамента рекомендую статью для выбора критерия проверки гипотез https://lit-review.ru/biostatistika/vybor-statisticheskogo-kriteriya/, https://statpsy.ru/ - информация по критериям проверки гипотез.

Визуализации данных для аналитиков очень важны, при помощи них легче доносить желаемые мысли, а для того, чтобы лучше отобразить, нужно понимать, какие вообще существуют визуализации, делюсь с тобой своей подборкой для разных случае жизни https://www.notion.so/d9392e1cb71f48cfb1fbf7f557a2b7e7 


SQL важный навык, который нужно поддерживать для этого существует много тренажёров:
* Статья на хабр с топ-тренажёрами: https://techrocks.ru/2019/11/22/7-sites-to-perfect-sql-query-writing-skills/
* задачки SQL - https://habr.com/ru/company/dcmiran/blog/500360/
* https://sqlzoo.net/


### Вывод

- `После 1 января 2000 года вышло 819 книг;`
- `На книгу Twilight (Twilight #1) вышло максимальное количество обзоров` - 7, - `рейтинг книги` - 3.7;
- `Издательство, выпустившее наибольшее число книг толще 50 страниц:` Penguin Books, они выпустили 42 книги;
- `Автор с самой высокой средней оценкой книг:` J.K. Rowling/Mary GrandPré с рейтингом 4.29;
- `Среднее количество обзоров от пользователей, которые поставили больше 50 оценок:` 24.33.